In [ ]:
import os
import difflib
import traceback
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.core.display import display, HTML
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.tools import tool
from imodulon_functions import *

In [ ]:
os.environ["OPENAI_API_KEY"] = input("OpenAI API Key: ")
llm = ChatOpenAI(model="gpt-4o", temperature=0)
# Define the prompt
with open("imodulon_chat_prompt.txt", "r", encoding="utf8") as file:
    imodulon_chat_prompt = file.read()

# Initialize the HelloWorld tool
tools = [
    learn_about_imodulons,
    find_closest_imodulon,
    find_closest_gene,
    find_closest_condition,
    get_genes_of_imodulons,
    get_condition_info,
    get_gene_info,
    get_imodulon_info,
    plot_gene_expression,
    plot_imodulon_activity,
    plot_all_imodulon_activities_for_condition,
    compare_gene_expression,
    compare_imodulon_activities,
    plot_dima,
    execute_python_code
]

# Bind the tools to the model
llm_with_tools = llm.bind_tools(tools)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", imodulon_chat_prompt),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])


agent = create_tool_calling_agent(llm_with_tools, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)
chat_history = []

In [ ]:
display(HTML("""
    <style>
        .output_wrapper, .output {
            height: auto !important;
            max-height: 1000px;  /* Adjust this value as needed */
            overflow-y: auto;
        }
    </style>
"""))

In [ ]:
print("Welcome to iModulon Chat. Type 'exit' 'quit' or 'q' to quit\n")
while True:
    user_input = input("Input: ")
    if user_input.lower() in ['exit','quit','q']:
        break

    # Prepare the input for the agent
    input_data = {
        "input": user_input,
        "chat_history": chat_history
    }

    # Run the agent
    response = agent_executor.invoke(input_data)
    print(f"\n{response['output']}\n")
    # Update the chat history
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response['output']))